In [1]:
import numpy as np
import otmol as otm
import pandas as pd
from openbabel import pybel
#import matplotlib.pyplot as plt
#import seaborn as sns
import os
#from rdkit.Chem import MolFromPDBFile
#from experiment_utils import interactive_alignment_plot_py3dmol, copy_and_rename_pdb_files
#from experiment_utils import get_transformation_matrix, find_assignment
# %matplotlib widget

In [ ]:
# directly calculate using pybel
for ligand in ['215d_DNA', '8w4x_BGCGLC', '8w4x_BGC', '6ln3_ATP', '5bvs_EIC', '4csv_imatinib', '1ln1_DLP']:
    data_path = "../Data/extra_bio_ligands"
    df = pd.read_csv(f'arbalign_output/{ligand}_results.csv')
    BCI = []
    mismatched_bond_indices = []
    for i in range(len(df)):
        path1 = os.path.join(data_path, 'extra_bio_ligands_reference', df.iloc[i]['nameA']+'.xyz')
        path2 = os.path.join('arbalign_output', df.iloc[i]['nameB']+'-aligned_to-'+df.iloc[i]['nameA']+'.xyz')
        molA = next(pybel.readfile("xyz", path1))
        molB = next(pybel.readfile("xyz", path2))
        X_A, T_A, B_A = otm.tl.process_molecule(molA, heavy_atoms_only=True) 
        X_B, T_B, B_B = otm.tl.process_molecule(molB, heavy_atoms_only=True)
        mismatched_bond_count, indices = otm.tl.mismatched_bond_counter(B_A, B_B, np.arange(X_A.shape[0]), B_A.shape[0], B_B.shape[0], only_A_bonds=True)
        BCI.append(mismatched_bond_count/np.sum(B_A)*200) # np.sum(B_A) = 2*the number of bonds in A
        mismatched_bond_indices.append(indices)
    df['BCI (%)'] = BCI
    df['mismatched bond indices'] = mismatched_bond_indices # BCI
    df.to_csv(f'arbalign_output/{ligand}_results_with_mismatched_bond.csv', index=False)

In [4]:
data_path = "../data/S1-MA-W1"
df = pd.read_csv(f'arbalign_output/S1MAW1_result.csv')
BCI = []
mismatched_bond_indices = []
for i in range(len(df)):
    path1 = os.path.join(data_path, df.iloc[i]['nameA']+'.xyz')
    path2 = os.path.join('arbalign_output', df.iloc[i]['nameB']+'-aligned_to-'+df.iloc[i]['nameA']+'.xyz')
    molA = next(pybel.readfile("xyz", path1))
    molB = next(pybel.readfile("xyz", path2))
    X_A, T_A, B_A = otm.tl.process_molecule(molA) 
    X_B, T_B, B_B = otm.tl.process_molecule(molB)
    mismatched_bond_count, indices = otm.tl.mismatched_bond_counter(B_A, B_B, np.arange(X_A.shape[0]), B_A.shape[0], B_B.shape[0], only_A_bonds=True)
    BCI.append(mismatched_bond_count/np.sum(B_A)*200) # np.sum(B_A) = 2*the number of bonds in A
    mismatched_bond_indices.append(indices)
df['BCI (%)'] = BCI
df['bond indices in A that are not in B'] = mismatched_bond_indices
df.to_csv(f'arbalign_output/S1MAW1_results_with_mismatched_bond.csv', index=False)

In [5]:
df = pd.read_csv('arbalign_output/4csv_imatinib_results_with_mismatched_bond.csv')
df.iloc[50]['nameB']
df.iloc[50]['mismatched bond indices']

'[(0, 5), (2, 3), (3, 4), (4, 5), (6, 7), (6, 11), (7, 8), (8, 9), (10, 11), (10, 12), (12, 13), (13, 18), (14, 36), (16, 17), (17, 18), (19, 20), (20, 21), (21, 22), (23, 24), (24, 25), (24, 27), (28, 29), (28, 34), (29, 30), (31, 32), (31, 33)]'

In [8]:
for ligand in ['6ln3_ATP', '5bvs_EIC', '8w4x_BGC', '4csv_imatinib', '215d_DNA', '8w4x_BGCGLC', '1ln1_DLP']:
    df = pd.read_csv(f'arbalign_output/{ligand}_results_with_mismatched_bond.csv')
    print(ligand,df['BCI (%)'].describe())


6ln3_ATP count    51.000000
mean     28.461081
std       3.836820
min      15.151515
25%      27.272727
50%      30.303030
75%      30.303030
max      39.393939
Name: BCI (%), dtype: float64
5bvs_EIC count    50.000000
mean     24.736842
std      15.672679
min       0.000000
25%      15.789474
50%      18.421053
75%      36.842105
max      52.631579
Name: BCI (%), dtype: float64
8w4x_BGC count    50.000000
mean     16.800000
std      20.547605
min       0.000000
25%       0.000000
50%       0.000000
75%      30.000000
max      70.000000
Name: BCI (%), dtype: float64
4csv_imatinib count    51.000000
mean     21.233859
std      16.202864
min       0.000000
25%       8.536585
50%      21.951220
75%      30.487805
max      63.414634
Name: BCI (%), dtype: float64
215d_DNA count    50.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: BCI (%), dtype: float64
8w4x_BGCGLC count    50.000000
mean     30.200000
std      17.407481
min       0

In [ ]:
"""
215 DNA has a strange bug, not sure why.
for ligand in ['1ln1_DLP', '5bvs_EIC']:
    data_path = "../Data/extra_bio_ligands"
    df = pd.read_csv(f'arbalign_output/{ligand}_results.csv')
    mismatched_bond = []
    for i in range(len(df)):
        target_path = os.path.join(data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', f'boltz_results_{ligand}', df.iloc[i]['nameB']+'.xyz')
        arbalign_path = os.path.join('arbalign_output', df.iloc[i]['nameB']+'-aligned_to-'+df.iloc[i]['nameA']+'.xyz')
        mol_target = next(pybel.readfile("xyz", target_path))
        mol_arbalign = next(pybel.readfile("xyz", arbalign_path))
        X_target, _, _ = otm.tl.process_molecule(mol_target, heavy_atoms_only=True)
        X_arbalign, _, _ = otm.tl.process_molecule(mol_arbalign, heavy_atoms_only=True)
        swap, reflect = df.iloc[i]['swap'], df.iloc[i]['reflect']
        swap = list(map(int, swap.replace('(', '').replace(')', '').split(',')))
        reflect = list(map(int, reflect.replace('(', '').replace(')', '').split(',')))
        X_arbalign = X_arbalign @ (np.linalg.inv(get_transformation_matrix(swap, reflect))).T
        assignment = find_assignment(X_arbalign, X_target)
        path1 = os.path.join(data_path, 'extra_bio_ligands_reference', df.iloc[i]['nameA']+'.pdb')
        path2 = os.path.join(
                data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', f'boltz_results_{ligand}', df.iloc[i]['nameB']+'.pdb'
            )  
        X_A, T_A, B_A = otm.tl.parse_pdb_file(path1)
        X_B, T_B, B_B = otm.tl.parse_pdb_file(path2)
        mismatched_bond.append(otm.tl.mismatched_bond_counter(B_A, B_B, assignment, B_A.shape[0], B_B.shape[0], only_look_at_bonds=True)/np.sum(B_A)*200)
    df['% mismatched bond'] = mismatched_bond
    df.to_csv(f'arbalign_output/{ligand}_results_with_mismatched_bond.csv', index=False)
"""

In [ ]:
"""
# first infer the assignment using arbalign output
data_path = "../Data/extra_bio_ligands"
for ligand in ['4csv_imatinib', '6ln3_ATP']:
    df = pd.read_csv(f'arbalign_output/{ligand}_results.csv')
    mismatched_bond = []
    for i in range(51):
        if i < 50:
            target_path = os.path.join(data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', f'boltz_results_{ligand}', df.iloc[i]['nameB']+'.xyz')
        if i == 50:
            target_path = os.path.join(data_path, 'extra_bio_ligands_CSD', df.iloc[i]['nameB']+'.xyz')
        arbalign_path = os.path.join('arbalign_output', df.iloc[i]['nameB']+'-aligned_to-'+df.iloc[i]['nameA']+'.xyz')
        mol_target = next(pybel.readfile("xyz", target_path))
        mol_arbalign = next(pybel.readfile("xyz", arbalign_path))
        X_target, _, _ = otm.tl.process_molecule(mol_target, heavy_atoms_only=True)
        X_arbalign, _, _ = otm.tl.process_molecule(mol_arbalign, heavy_atoms_only=True)
        swap, reflect = df.iloc[i]['swap'], df.iloc[i]['reflect']
        swap = list(map(int, swap.replace('(', '').replace(')', '').split(',')))
        reflect = list(map(int, reflect.replace('(', '').replace(')', '').split(',')))
        X_arbalign = X_arbalign @ (np.linalg.inv(get_transformation_matrix(swap, reflect))).T
        assignment = find_assignment(X_arbalign, X_target)
        path1 = os.path.join(data_path, 'extra_bio_ligands_reference', df.iloc[i]['nameA']+'.pdb')
        if i < 50:
            path2 = os.path.join(
                data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', f'boltz_results_{ligand}', df.iloc[i]['nameB']+'.pdb'
            )  
        if i == 50:
            path2 = os.path.join(
                data_path, 'extra_bio_ligands_CSD', df.iloc[i]['nameB']+'.pdb'
            )  
        X_A, T_A, B_A = otm.tl.parse_pdb_file(path1)
        X_B, T_B, B_B = otm.tl.parse_pdb_file(path2)
        mismatched_bond.append(otm.tl.mismatched_bond_counter(B_A, B_B, assignment, B_A.shape[0], B_B.shape[0], only_look_at_bonds=True)/np.sum(B_A)*200)
    df['% mismatched bond'] = mismatched_bond
    df.to_csv(f'arbalign_output/{ligand}_results_with_mismatched_bond.csv', index=False)
"""

In [ ]:
"""
data_path = "../Data/extra_bio_ligands"
for ligand in ['8w4x_BGCGLC', '8w4x_BGC']:
    df = pd.read_csv(f'arbalign_output/{ligand}_results.csv')
    mismatched_bond = []
    for i in range(len(df)):
        target_path = os.path.join(data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', 'boltz_results_8w4x_sugars', df.iloc[i]['nameB']+'.xyz')
        arbalign_path = os.path.join('arbalign_output', df.iloc[i]['nameB']+'-aligned_to-'+df.iloc[i]['nameA']+'.xyz')
        mol_target = next(pybel.readfile("xyz", target_path))
        mol_arbalign = next(pybel.readfile("xyz", arbalign_path))
        X_target, _, _ = otm.tl.process_molecule(mol_target, heavy_atoms_only=True)
        X_arbalign, _, _ = otm.tl.process_molecule(mol_arbalign, heavy_atoms_only=True)
        swap, reflect = df.iloc[i]['swap'], df.iloc[i]['reflect']
        swap = list(map(int, swap.replace('(', '').replace(')', '').split(',')))
        reflect = list(map(int, reflect.replace('(', '').replace(')', '').split(',')))
        #X_target = X_target @ (get_transformation_matrix(swap, reflect)).T
        X_arbalign = X_arbalign @ (np.linalg.inv(get_transformation_matrix(swap, reflect))).T
        assignment = find_assignment(X_arbalign, X_target)
        path1 = os.path.join(data_path, 'extra_bio_ligands_reference', df.iloc[i]['nameA']+'.pdb')
        path2 = os.path.join(
                data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', 'boltz_results_8w4x_sugars', df.iloc[i]['nameB']+'.pdb'
            )  
        X_A, T_A, B_A = otm.tl.parse_pdb_file(path1)
        X_B, T_B, B_B = otm.tl.parse_pdb_file(path2)
        mismatched_bond.append(otm.tl.mismatched_bond_counter(B_A, B_B, assignment, B_A.shape[0], B_B.shape[0], only_look_at_bonds=True)/np.sum(B_A)*200)
    df['% mismatched bond'] = mismatched_bond
    df.to_csv(f'arbalign_output/{ligand}_results_with_mismatched_bond.csv', index=False)
"""

In [ ]:
"""
ligand = '215d_DNA'
data_path = "../Data/extra_bio_ligands"
df = pd.read_csv(f'arbalign_output/{ligand}_results.csv')
mismatched_bond = []
for i in range(len(df)):
    target_path = os.path.join(data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', 'AF3_215D_DNA_chainBC', df.iloc[i]['nameB']+'.xyz')
    arbalign_path = os.path.join('arbalign_output', df.iloc[i]['nameB']+'-aligned_to-'+df.iloc[i]['nameA']+'.xyz')
    mol_target = next(pybel.readfile("xyz", target_path))
    mol_arbalign = next(pybel.readfile("xyz", arbalign_path))
    X_target, _, _ = otm.tl.process_molecule(mol_target, heavy_atoms_only=True)
    X_arbalign, _, _ = otm.tl.process_molecule(mol_arbalign, heavy_atoms_only=True)
    swap, reflect = df.iloc[i]['swap'], df.iloc[i]['reflect']
    swap = list(map(int, swap.replace('(', '').replace(')', '').split(',')))
    reflect = list(map(int, reflect.replace('(', '').replace(')', '').split(',')))
    #X_target = X_target @ (get_transformation_matrix(swap, reflect)).T
    X_arbalign = X_arbalign @ (np.linalg.inv(get_transformation_matrix(swap, reflect))).T
    #print(X_arbalign, X_target)
    assignment = find_assignment(X_arbalign, X_target)
    path1 = os.path.join(data_path, 'extra_bio_ligands_reference', df.iloc[i]['nameA']+'.pdb')
    path2 = os.path.join(
            data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', 'AF3_215D_DNA_chainBC', df.iloc[i]['nameB']+'.pdb'
        )  
    X_A, T_A, B_A = otm.tl.parse_pdb_file(path1)
    X_B, T_B, B_B = otm.tl.parse_pdb_file(path2)
    mismatched_bond.append(otm.tl.mismatched_bond_counter(B_A, B_B, assignment, B_A.shape[0], B_B.shape[0], only_look_at_bonds=True)/np.sum(B_A)*200)

df['% mismatched bond'] = mismatched_bond
df.to_csv(f'arbalign_output/{ligand}_results_with_mismatched_bond.csv', index=False)
"""